<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Notebook: Migration for Custom Monitor Insntaces


Target Version: Cloud pak for Data **v4.5.1**

The user needs to provide the necessary inputs (where marked) to be able to proceed. 

**Note**: This notebook is designed to migrate some monitor instaces which is created before Cloud pak for data 4.5.0. Without this procedure, you cannot have successful evaluation for custom monitor in model risk management screen.

**Contents:**
1. [Package Installation](#Package-installation)
2. [User Inputs](#User-Inputs)
3. [Setting up Services](#Setting-up-Services)
4. [Collect resources to be migrated](#Collect-resources-to-be-migrated)
5. [Migrate monitor instances](#Migrate-monitor-instances)
6. [Confirm the result](#Confirm-the-result)

## Package installation

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

Action: restart the kernel if you executed above cell!

## User Inputs

The following inputs are required:

1. **IBM_CPD_ENDPOINT:** The URL representing the IBM Cloud Pak for Data service endpoint.
2. **IBM_CPD_USERNAME:** IBM Cloud Pak for Data username used to obtain a bearer token.
3. **IBM_CPD_PASSWORD:** IBM Cloud Pak for Data password used to obtain a bearer token.
4. **CPD_API_KEY:** IBM Cloud Pak for Data API Key used to obtain a bearer token.

In [ ]:
# IBM Cloud Pak for Data credentials
IBM_CPD_ENDPOINT = "<The URL representing the IBM Cloud Pak for Data service endpoint.>"
IBM_CPD_USERNAME = "<IBM Cloud Pak for Data username used to obtain a bearer token.>"
IBM_CPD_PASSWORD = "<IBM Cloud Pak for Data password used to obtain a bearer token.>"
SERVICE_INSTANCE_ID = "<SERVICE_INSTANCE_ID>" #Default is 00000000-0000-0000-0000-000000000000
CPD_API_KEY ="<API KEY>" 

## Setting up Services

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

service_credentials = {
                "url": IBM_CPD_ENDPOINT,
                "username": IBM_CPD_USERNAME,
                "password": IBM_CPD_PASSWORD,
                # "apikey": CPD_API_KEY
                }

authenticator = CloudPakForDataAuthenticator(
        url=service_credentials['url'],
        username=service_credentials['username'],
        password=service_credentials['password'],
        # apikey=service_credentials['apikey'],
        disable_ssl_verification=True
)

client = APIClient(
    service_url=service_credentials['url'],
    service_instance_id=SERVICE_INSTANCE_ID,
    authenticator=authenticator
)

print(client.version)

In [ ]:
OOTB_DEFINITONS = [
    'assurance',
    'fairness',
    'performance',
    'explainability',
    'mrm',
    'correlations',
    'drift',
    'quality'
]

PATCH_DOCUMENT = [
    {
        "op": "remove",
        "path": "/schedule"
    },
    {
        "op": "remove",
        "path": "/schedule_id"
    }
]

## Collect resources to be migrated

In [ ]:
# Explore custom monitor definitions which should not have schedule
definitions = client.monitor_definitions.list().result
custom_monitor_def_ids = [
    i.metadata.id for i in definitions.monitor_definitions if (not i.metadata.id in OOTB_DEFINITONS) and (not "monitor_runtime" in i.entity.to_dict())
]
print(custom_monitor_def_ids)

In [ ]:
# Collect monitor instances to be patched
monitor_instances_to_be_patched = client.monitor_instances.list(
    monitor_definition_id=",".join(custom_monitor_def_ids),
    target_target_type="subscription"
).result.monitor_instances

print("The count of patched monitor instances is {}".format(len(monitor_instances_to_be_patched)))

for i in monitor_instances_to_be_patched:
    print("definition id: {}, monitor instance id: {}".format(i.entity.monitor_definition_id, i.metadata.id))
    print("schedule id: {}, schedule: {}".format(i.entity.schedule_id, i.entity.schedule))

## Migrate monitor instances

In [ ]:
for mi in monitor_instances_to_be_patched:
    client.monitor_instances.update(
        monitor_instance_id=mi.metadata.id,
        patch_document=PATCH_DOCUMENT
    )

## Confirm the result 

In [ ]:
# Confirm whether schedule and its id are deleted properly
monitor_instances_to_be_patched = client.monitor_instances.list(
    monitor_definition_id=",".join(custom_monitor_def_ids),
    target_target_type="subscription"
).result.monitor_instances

print("The count of patched monitor instances is {}".format(len(monitor_instances_to_be_patched)))

for i in monitor_instances_to_be_patched:
    print("definition id: {}, monitor instance id: {}".format(i.entity.monitor_definition_id, i.metadata.id))
    print("schedule id: {}, schedule: {}".format(i.entity.schedule_id, i.entity.schedule))
# Expected print will be: 
    # definition id: <some custom monitor id>, monitor instance id: <some guid for monitor instance>
    # schedule id: None, schedule: None    